In [58]:
import pandas as pd
import numpy as np
import pyodbc
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [59]:
svr_name = 'KU1ICDDWV011.kunet.ae'
db_name = 'CACM'
u_name = 'cacm_user'
u_pass = 'Welcome#123'

In [60]:
eng = create_engine("mssql+pyodbc://"+u_name+":"+u_pass+"@"+svr_name+"/"+db_name+"?driver=ODBC+Driver+17+for+SQL+Server")

In [61]:
query_resigned = "select * from ADERP.dbo.XXDOF_BI_KU_Resignation_T"
query_payroll = "select * from ADERP.dbo.ku_payroll_latest"

In [62]:
resigned = pd.read_sql(query_resigned, eng)
payroll = pd.read_sql(query_payroll,eng)

In [63]:
resigned.columns

Index(['PERSON_ID', 'PERSON_EXTRA_INFO_ID', 'EMPLOYEE_NUMBER',
       'LEAVING_REASON', 'NOTIFY_ON', 'ACTUAL_TERMINATION_DATE', 'REMARKS',
       'CREATION_DATE', 'LASTUPDATEDATE1'],
      dtype='object')

In [64]:
payroll.columns

Index(['EmployeeNumber', 'PersonID', 'EmployeeGrade', 'PayYear', 'PayMonth',
       'PayrollName', 'Basic', 'CoLA', 'MarketDiff', 'Vacation', 'ChildAllow',
       'AdminStipend', 'ActingAllow', 'OtherLegacy', 'Gross'],
      dtype='object')

In [65]:
resigned_data = resigned.merge(payroll,left_on='EMPLOYEE_NUMBER',right_on='EmployeeNumber',how='left')

In [66]:
resigned_data = resigned_data[['EMPLOYEE_NUMBER','ACTUAL_TERMINATION_DATE','PayrollName','PayYear','PayMonth','Gross']]

In [67]:
resigned_data.columns

Index(['EMPLOYEE_NUMBER', 'ACTUAL_TERMINATION_DATE', 'PayrollName', 'PayYear',
       'PayMonth', 'Gross'],
      dtype='object')

In [68]:
resigned_data['Last'] = resigned_data['PayMonth']+'-'+resigned_data['PayYear']

In [69]:
resigned_data['Termination'] = [x[3:] for x in resigned_data['ACTUAL_TERMINATION_DATE']]

In [70]:
resigned_data = resigned_data.astype({'Last':'datetime64','Termination':'datetime64'})

In [71]:
resigned_data['Diff'] = resigned_data['Termination'] <= resigned_data['Last']

In [72]:
resigned_data['Diff'] = resigned_data['Diff'].astype('int')

In [75]:
resigned_data[resigned_data['Diff'] == 1].to_excel('termination.xlsx')

In [77]:
resigned_data.drop(['PayrollName'],axis='columns',inplace=True)

In [78]:
resigned_data

,EMPLOYEE_NUMBER,ACTUAL_TERMINATION_DATE,PayYear,PayMonth,Gross,Last,Termination,Diff
0,KU524,20-DEC-2021,2021,NOV,63059.0,2021-11-01,2021-12-01,0
1,KU533,01-NOV-2021,2021,SEP,24994.0,2021-09-01,2021-11-01,0
2,KU700,08-DEC-2021,2022,MAR,32831.0,2022-03-01,2021-12-01,1
3,KU500423,07-OCT-2021,2021,SEP,13700.0,2021-09-01,2021-10-01,0
4,KU150,23-DEC-2021,2021,NOV,36523.0,2021-11-01,2021-12-01,0
...,...,...,...,...,...,...,...,...
366,KU271,16-JAN-2022,2021,DEC,29356.0,2021-12-01,2022-01-01,0
367,KU586,28-FEB-2022,2022,JAN,51733.0,2022-01-01,2022-02-01,0
368,KU500068,08-DEC-2021,2021,NOV,12548.0,2021-11-01,2021-12-01,0
369,KU500279,30-NOV-2021,2021,NOV,22000.0,2021-11-01,2021-11-01,1
